In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pulsar-classification-for-class-prediction/Pulsar.csv
/kaggle/input/playground-series-s3e10/sample_submission.csv
/kaggle/input/playground-series-s3e10/train.csv
/kaggle/input/playground-series-s3e10/test.csv
/kaggle/input/predicting-pulsar-starintermediate/pulsar_data_train.csv
/kaggle/input/predicting-pulsar-starintermediate/pulsar_data_test.csv


In [3]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e10/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e10/test.csv')
back_train = pd.read_csv('/kaggle/input/pulsar-classification-for-class-prediction/Pulsar.csv')
back_train.head(2)
df_train['is_generated'] = 1
df_test['is_generated'] = 1
back_train['is_generated'] = 0
back_train.head(2)

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve,Class,is_generated
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0,0


In [4]:
def remove_outlier(train):
    outliers = train[
        (train['Class']==1)&
        (train['Mean_Integrated']>115)&
        (train['SD']>45)&
        (train['EK']<0.03)&
        (train['Skewness']<1)&
        (train['Mean_DMSNR_Curve']<20)
    ].index
    train.drop(outliers,inplace=True)
    train.reset_index()
    return train


In [5]:
X = df_train.drop(columns=['id'])
train_data = pd.concat([X, back_train],axis=0)
train_data = remove_outlier(train_data)
X = train_data.drop(columns=['Class'])
y = train_data['Class']
test = df_test.drop(columns=['id'])

In [ ]:
def f_importance_plot(f_imp):
    fig = plt.figure(figsize = (15, 0.35*len(f_imp)))
    plt.title('Feature importances', size=25, y=1.05, 
              fontname='Calibri', fontweight='bold', color='#444444')
    a = sns.barplot(data=f_imp, x='avg_imp', y='feature', 
                    palette='Blues_d', linestyle="-", 
                    linewidth=1, edgecolor="black")
    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('')
    plt.yticks(size=11, color='#444444')
    
    for j in ['right', 'top', 'bottom']:
        a.spines[j].set_visible(False)
    for j in ['left']:
        a.spines[j].set_linewidth(0.5)
    plt.show()

In [ ]:
from itertools import combinations
from sklearn.preprocessing import StandardScaler
class Preprocessor:
    def __init__(self, numeric_columns=None, max_pattern=2):
        self.numeric_columns = numeric_columns
        self.max_pattern = max_pattern
        self.scaler = None
        
    def preprocess(self, X_train, X_test):
        
        X_train = self.create_numeric_combinations(X_train)
        X_test = self.create_numeric_combinations(X_test)
        
        numeric_columns = [_ for _ in X_train.columns if X_train[_].dtype == 'float']
        scaler = StandardScaler()
        X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
        X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

        return X_train, X_test
    
    def create_numeric_combinations(self, df):
        new_cols = []
        for comb in range(2, len(self.numeric_columns) + 1):
            for col in combinations(self.numeric_columns, comb):
                if len(col) > self.max_pattern:
                    break
                col_names = list(col)
                new_col = '_'.join(col_names) + '_mult'
                df[new_col] = df[col_names[0]]
                for c in col_names[1:]:
                    df[new_col] *= df[c]
                new_cols.append(new_col)

        return df

numeric_columns = [_ for _ in test.columns if 'is_generated' not in _]
pp = Preprocessor(numeric_columns)
X, test = pp.preprocess(X, test)
print(f"X_train shape :{X.shape}", f"X_test shape :{test.shape}")

In [2]:

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
from umap import UMAP
from sklearn.cluster import KMeans
from haversine import haversine

import optuna
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss

lgb_params = {
    'max_depth': 9,
    'learning_rate': 0.01,
    'min_data_in_leaf': 36, 
    'num_leaves': 100, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.89, 
    'bagging_freq': 5, 
    'lambda_l2': 28,
    
    'seed': 3407,
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'device': 'cpu', 
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}

f_imp = pd.DataFrame({'feature': X.columns})
FOLDS = 10
seed = 3407
bold = ['\033[1m', '\033[0m']
temp_pred = []
temp_y = []
logs = [] 
cv_scores = []
predictions =[] 
k = KFold(n_splits=FOLDS, random_state=seed, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(k.split(X, y)):
    print(f'\n--- FOLD {fold+1} ---')
    model = lgb.LGBMClassifier(**lgb_params)
    model = model.fit(X.iloc[train_idx],y.iloc[train_idx])
    x_test,y_test = X.iloc[val_idx],y.iloc[val_idx]
    fold_xt_pred = model.predict_proba(x_test)[:,1]
    fold_test_pred = model.predict(test)
    temp_pred.append(fold_xt_pred)
    temp_y.append(y_test)
    logs.append(log_loss(y_test,fold_xt_pred))
    print("--------------", log_loss(y_test,fold_xt_pred))
    predictions.append(fold_test_pred)
    cv_scores.append(np.mean(logs))
lgbm_cv_score = np.mean(cv_scores)
print('LGBM CV SCORE : ', lgbm_cv_score)

NameError: name 'pd' is not defined

In [7]:
predictions

[array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1])]

In [ ]:
ans_lgb = pd.read_csv("/kaggle/input/playground-series-s3e10/sample_submission.csv")
ans_lgb["Class"] = predictions #np.where(predictions>0.5, 1, 0)
ans_lgb.to_csv("ans_lgb.csv", index=False)

In [9]:
test

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve,is_generated
0,140.046875,54.507800,0.058862,-0.567263,2.337793,14.868335,9.591760,117.988781,1
1,107.828125,51.578965,0.284368,-0.338430,1.574415,12.501437,11.694968,182.704822,1
2,135.062500,49.812343,-0.087784,-0.094341,3.576923,21.243336,7.252386,59.021499,1
3,112.812500,41.926647,0.519921,1.287762,6.669732,29.013153,5.097661,27.105240,1
4,96.210938,35.322620,0.481286,2.443080,2.218227,17.041064,9.766006,117.131775,1
...,...,...,...,...,...,...,...,...,...
78372,119.328125,46.449223,0.227636,0.161986,1.120401,13.883072,14.153556,213.485463,1
78373,125.835938,43.986678,0.078460,0.077917,2.342809,13.008583,9.907167,136.815454,1
78374,131.789062,43.493074,0.049140,0.402859,2.116221,14.639555,10.898768,147.929016,1
78375,79.476562,42.579993,0.817383,1.818474,1.903010,15.844982,10.723755,143.273847,1


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Creating Models

In [8]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
             nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x
    
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.233197  [   64/60000]
loss: 0.320735  [ 6464/60000]
loss: 0.204201  [12864/60000]
loss: 0.295273  [19264/60000]
loss: 0.254464  [25664/60000]
loss: 0.354950  [32064/60000]
loss: 0.263224  [38464/60000]
loss: 0.342192  [44864/60000]
loss: 0.318746  [51264/60000]
loss: 0.240935  [57664/60000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.366559 

Epoch 2
-------------------------------
loss: 0.233197  [   64/60000]
loss: 0.320735  [ 6464/60000]
loss: 0.204201  [12864/60000]
loss: 0.295273  [19264/60000]
loss: 0.254464  [25664/60000]
loss: 0.354950  [32064/60000]
loss: 0.263224  [38464/60000]
loss: 0.342192  [44864/60000]
loss: 0.318746  [51264/60000]
loss: 0.240935  [57664/60000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.366559 

Epoch 3
-------------------------------
loss: 0.233197  [   64/60000]
loss: 0.320735  [ 6464/60000]
loss: 0.204201  [12864/60000]
loss: 0.295273  [19264/60000]
loss: 0.254464  [25664/60000]
loss: 0.354950  [32064/600

In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [17]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>